In [1]:
!pip install numdifftools
!pip install git+https://github.com/Shopify/bevel.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 1.9 MB/s eta 0:00:00
  Cloning https://github.com/Shopify/bevel.git to /tmp/pip-req-build-rjpj42z9
  Running command git clone --filter=blob:none --quiet https://github.com/Shopify/bevel.git /tmp/pip-req-build-rjpj42z9
  Resolved https://github.com/Shopify/bevel.git to commit 20767b2ab2c2843b71de986ee8c2608c26ba8ee7
  Preparing metadata (setup.py) ... done
  Created wheel for bevel: filename=bevel-0.1.0-py3-none-any.whl size=11977 sha256=ec0239c637c5fa118a739325614a28528bca3e924e1db4bd9d947372e7bbb296
  Stored in directory: /tmp/pip-ephem-wheel-cache-8p952uv2/wheels/78/87/64/4b96ca4512b4ee27a42df33c653a389aa887789c23e28d741a
Successfully built bevel


In [2]:
import pandas as pd
import numpy as np
from bevel.linear_ordinal_regression import OrderedLogit

In [3]:
season19_20 = pd.read_excel('https://drive.google.com/uc?export=download&id=11dqaaBjym0SYdkHt2OrchrN-ih2IqTSC')
season19_20 = season19_20[["HomeTeam","AwayTeam","FTR","B365H","B365D","B365A"]]
season19_20.columns = ['home_team', 'away_team', 'ftr', 'b365_h', 'b365_d', 'b365_a']
season19_20

,home_team,away_team,ftr,b365_h,b365_d,b365_a
0,Liverpool,Norwich,H,1.14,10.00,19.00
1,West Ham,Man City,A,12.00,6.50,1.22
2,Bournemouth,Sheffield United,D,1.95,3.60,3.60
3,Burnley,Southampton,H,2.62,3.20,2.75
4,Crystal Palace,Everton,D,3.00,3.25,2.37
...,...,...,...,...,...,...
375,Leicester,Man United,A,3.25,3.50,2.20
376,Man City,Norwich,H,1.08,11.00,26.00
377,Newcastle,Liverpool,A,7.50,5.25,1.36
378,Southampton,Sheffield United,H,2.15,3.50,3.30


In [4]:
season19_20.describe()

,b365_h,b365_d,b365_a
count,380.000000,380.000000,380.000000
mean,2.947921,4.301474,4.707632
std,2.235950,1.518987,4.266324
min,1.070000,3.000000,1.140000
25%,1.610000,3.400000,2.200000
50%,2.250000,3.750000,3.250000
75%,3.300000,4.750000,5.500000
max,15.000000,13.000000,26.000000


In [5]:
season19_20['b365_pr_h']= 1/(season19_20['b365_h'])/(1/(season19_20['b365_h'])+ 1/(season19_20['b365_d'])+ 1/(season19_20['b365_a']))
season19_20['b365_pr_d']= 1/(season19_20['b365_d'])/(1/(season19_20['b365_h'])+ 1/(season19_20['b365_d'])+ 1/(season19_20['b365_a']))
season19_20['b365_pr_a']= 1/(season19_20['b365_a'])/(1/(season19_20['b365_h'])+ 1/(season19_20['b365_d'])+ 1/(season19_20['b365_a']))

season19_20['b365_res']= np.where((season19_20['b365_h']<season19_20['b365_d']) & (season19_20['b365_h']<=season19_20['b365_a']),'H',\
                            np.where((season19_20['b365_d']<season19_20['b365_h']) & (season19_20['b365_d']<season19_20['b365_a']),'D',\
                                    np.where((season19_20['b365_a']<season19_20['b365_h']) & (season19_20['b365_a']<season19_20['b365_d']),'A',"")))

season19_20.drop(['b365_h', 'b365_d', 'b365_a'], axis=1, inplace=True)

season19_20

,home_team,away_team,ftr,b365_pr_h,b365_pr_d,b365_pr_a,b365_res
0,Liverpool,Norwich,H,0.851789,0.097104,0.051107,H
1,West Ham,Man City,A,0.078851,0.145570,0.775579,A
2,Bournemouth,Sheffield United,D,0.480000,0.260000,0.260000,H
3,Burnley,Southampton,H,0.360818,0.295420,0.343762,H
4,Crystal Palace,Everton,D,0.313588,0.289466,0.396947,A
...,...,...,...,...,...,...,...
375,Leicester,Man United,A,0.293613,0.272641,0.433746,A
376,Man City,Norwich,H,0.877408,0.086146,0.036446,H
377,Newcastle,Liverpool,A,0.125893,0.179847,0.694261,A
378,Southampton,Sheffield United,H,0.441345,0.271112,0.287543,H


In [6]:
tm_values = pd.read_excel('https://drive.google.com/uc?export=download&id=1owMZJpphIjNQUlFUlzgT9Zvm0Mx8Aqrg')
tm_values

,Team,TM Value
0,Man City,1270
1,Liverpool,1090
2,Tottenham,1040
3,Chelsea,797
4,Man United,775
5,Arsenal,680
6,Everton,525
7,Leicester,384
8,West Ham,371
9,Wolves,315


In [7]:
tm_values["home_team"] = tm_values["Team"]
tm_values["away_team"] = tm_values["Team"]
season19_20 = pd.merge(season19_20,tm_values[['home_team','TM Value']], on='home_team', how = 'left').rename(columns={'TM Value':'tm_h'})
season19_20 = pd.merge(season19_20,tm_values[['away_team','TM Value']], on='away_team', how = 'left').rename(columns={'TM Value':'tm_a'})
season19_20

,home_team,away_team,ftr,b365_pr_h,b365_pr_d,b365_pr_a,b365_res,tm_h,tm_a
0,Liverpool,Norwich,H,0.851789,0.097104,0.051107,H,1090,110
1,West Ham,Man City,A,0.078851,0.145570,0.775579,A,371,1270
2,Bournemouth,Sheffield United,D,0.480000,0.260000,0.260000,H,311,110
3,Burnley,Southampton,H,0.360818,0.295420,0.343762,H,205,259
4,Crystal Palace,Everton,D,0.313588,0.289466,0.396947,A,248,525
...,...,...,...,...,...,...,...,...,...
375,Leicester,Man United,A,0.293613,0.272641,0.433746,A,384,775
376,Man City,Norwich,H,0.877408,0.086146,0.036446,H,1270,110
377,Newcastle,Liverpool,A,0.125893,0.179847,0.694261,A,312,1090
378,Southampton,Sheffield United,H,0.441345,0.271112,0.287543,H,259,110


In [8]:
season19_20['log_tm_ratio'] = np.log(season19_20['tm_h']/season19_20['tm_a'])
season19_20

,home_team,away_team,ftr,b365_pr_h,b365_pr_d,b365_pr_a,b365_res,tm_h,tm_a,log_tm_ratio
0,Liverpool,Norwich,H,0.851789,0.097104,0.051107,H,1090,110,2.293453
1,West Ham,Man City,A,0.078851,0.145570,0.775579,A,371,1270,-1.230570
2,Bournemouth,Sheffield United,D,0.480000,0.260000,0.260000,H,311,110,1.039313
3,Burnley,Southampton,H,0.360818,0.295420,0.343762,H,205,259,-0.233818
4,Crystal Palace,Everton,D,0.313588,0.289466,0.396947,A,248,525,-0.749970
...,...,...,...,...,...,...,...,...,...,...
375,Leicester,Man United,A,0.293613,0.272641,0.433746,A,384,775,-0.702220
376,Man City,Norwich,H,0.877408,0.086146,0.036446,H,1270,110,2.446292
377,Newcastle,Liverpool,A,0.125893,0.179847,0.694261,A,312,1090,-1.250930
378,Southampton,Sheffield United,H,0.441345,0.271112,0.287543,H,259,110,0.856348


In [9]:
season19_20['win_value'] = np.where(season19_20['ftr'] == 'H', 0 ,(np.where(season19_20['ftr'] == 'D', 1, 2)))
season19_20

,home_team,away_team,ftr,b365_pr_h,b365_pr_d,b365_pr_a,b365_res,tm_h,tm_a,log_tm_ratio,win_value
0,Liverpool,Norwich,H,0.851789,0.097104,0.051107,H,1090,110,2.293453,0
1,West Ham,Man City,A,0.078851,0.145570,0.775579,A,371,1270,-1.230570,2
2,Bournemouth,Sheffield United,D,0.480000,0.260000,0.260000,H,311,110,1.039313,1
3,Burnley,Southampton,H,0.360818,0.295420,0.343762,H,205,259,-0.233818,0
4,Crystal Palace,Everton,D,0.313588,0.289466,0.396947,A,248,525,-0.749970,1
...,...,...,...,...,...,...,...,...,...,...,...
375,Leicester,Man United,A,0.293613,0.272641,0.433746,A,384,775,-0.702220,2
376,Man City,Norwich,H,0.877408,0.086146,0.036446,H,1270,110,2.446292,0
377,Newcastle,Liverpool,A,0.125893,0.179847,0.694261,A,312,1090,-1.250930,2
378,Southampton,Sheffield United,H,0.441345,0.271112,0.287543,H,259,110,0.856348,0


In [11]:
import numpy as np
np.int = int
# Prepare the train set
train_set = season19_20[:200]

# Fit the model
ol = OrderedLogit()
ol.fit(train_set['log_tm_ratio'], train_set['win_value'])

# Print the model coefficients
print(f'beta = {ol.coef_[0]:.4f}')
print(f'intercept_home_draw = {ol.coef_[1]:.4f}')
print(f'intercept_draw_away = {ol.coef_[2]:.4f}')

beta = -0.6417
intercept_home_draw = -0.3216
intercept_draw_away = 0.7911


In [12]:
season19_20['tm_pred_h'] = 1/(1+np.exp(-(ol.coef_[1]-ol.coef_[0]*season19_20['log_tm_ratio'])))
season19_20['tm_pred_d'] = 1/(1+np.exp(-(ol.coef_[2]-ol.coef_[0]*season19_20['log_tm_ratio']))) - season19_20['tm_pred_h']
season19_20['tm_pred_a'] = 1 - season19_20['tm_pred_h'] - season19_20['tm_pred_d']
season19_20.head()

,home_team,away_team,ftr,b365_pr_h,b365_pr_d,b365_pr_a,b365_res,tm_h,tm_a,log_tm_ratio,win_value,tm_pred_h,tm_pred_d,tm_pred_a
0,Liverpool,Norwich,H,0.851789,0.097104,0.051107,H,1090,110,2.293453,0,0.759526,0.146218,0.094256
1,West Ham,Man City,A,0.078851,0.145570,0.775579,A,371,1270,-1.230570,2,0.247647,0.252718,0.499636
2,Bournemouth,Sheffield United,D,0.480000,0.260000,0.260000,H,311,110,1.039313,1,0.585485,0.225740,0.188775
3,Burnley,Southampton,H,0.360818,0.295420,0.343762,H,205,259,-0.233818,0,0.384235,0.270756,0.345009
4,Crystal Palace,Everton,D,0.313588,0.289466,0.396947,A,248,525,-0.749970,1,0.309424,0.267424,0.423152


In [13]:
season19_20['maxprob'] =season19_20[['tm_pred_h','tm_pred_d','tm_pred_a']].max(axis=1)
season19_20['tm_res']=np.where(season19_20['maxprob']==season19_20['tm_pred_a'],'A',\
                               np.where(season19_20['maxprob']==season19_20['tm_pred_d'],'D','H'))
season19_20['tm_true']= np.where(season19_20['tm_res'] == season19_20['ftr'],1,0)
season19_20['b365_true']= np.where(season19_20['b365_res'] == season19_20['ftr'],1,0)

season19_20.drop(['maxprob'], axis=1, inplace=True)

season19_20.head()

,home_team,away_team,ftr,b365_pr_h,b365_pr_d,b365_pr_a,b365_res,tm_h,tm_a,log_tm_ratio,win_value,tm_pred_h,tm_pred_d,tm_pred_a,tm_res,tm_true,b365_true
0,Liverpool,Norwich,H,0.851789,0.097104,0.051107,H,1090,110,2.293453,0,0.759526,0.146218,0.094256,H,1,1
1,West Ham,Man City,A,0.078851,0.145570,0.775579,A,371,1270,-1.230570,2,0.247647,0.252718,0.499636,A,1,1
2,Bournemouth,Sheffield United,D,0.480000,0.260000,0.260000,H,311,110,1.039313,1,0.585485,0.225740,0.188775,H,0,0
3,Burnley,Southampton,H,0.360818,0.295420,0.343762,H,205,259,-0.233818,0,0.384235,0.270756,0.345009,H,1,1
4,Crystal Palace,Everton,D,0.313588,0.289466,0.396947,A,248,525,-0.749970,1,0.309424,0.267424,0.423152,A,0,0


In [14]:
test_set = season19_20[200:].copy()
test_set.describe()

,b365_pr_h,b365_pr_d,b365_pr_a,tm_h,tm_a,log_tm_ratio,win_value,tm_pred_h,tm_pred_d,tm_pred_a,tm_true,b365_true
count,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000
mean,0.428786,0.240324,0.330890,474.861111,475.677778,0.001039,0.805556,0.427646,0.245427,0.326927,0.516667,0.511111
std,0.202534,0.052468,0.186172,339.088455,341.764574,1.022250,0.852844,0.148391,0.028536,0.135297,0.501116,0.501271
min,0.072800,0.086146,0.036446,110.000000,110.000000,-2.446292,0.000000,0.131096,0.136810,0.086210,0.000000,0.000000
25%,0.279673,0.215863,0.187309,248.000000,242.250000,-0.739064,0.000000,0.310925,0.234292,0.222850,0.000000,0.000000
50%,0.413264,0.252640,0.297327,315.000000,313.500000,0.031216,1.000000,0.425180,0.254632,0.307653,1.000000,1.000000
75%,0.572410,0.279934,0.451063,703.750000,703.750000,0.713854,2.000000,0.534069,0.267157,0.421446,1.000000,1.000000
max,0.877408,0.317905,0.815859,1270.000000,1270.000000,2.446292,2.000000,0.776981,0.271201,0.685388,1.000000,1.000000


In [15]:
pd.crosstab(test_set['ftr'], test_set['tm_res'],dropna= True)

tm_res,A,H
ftr,,
A,28,23
D,13,30
H,21,65


In [16]:
pd.crosstab(test_set['ftr'], test_set['b365_res'],dropna= True)

b365_res,A,H
ftr,,
A,29,22
D,15,28
H,23,63


# Brier Score

In [17]:
test_set['home_value']= np.where(test_set['ftr']=='H',1,0)
test_set['draw_value']= np.where(test_set['ftr']=='D',1,0)
test_set['away_value']= np.where(test_set['ftr']=='A',1,0)
test_set

,home_team,away_team,ftr,b365_pr_h,b365_pr_d,b365_pr_a,b365_res,tm_h,tm_a,log_tm_ratio,win_value,tm_pred_h,tm_pred_d,tm_pred_a,tm_res,tm_true,b365_true,home_value,draw_value,away_value
200,Burnley,Aston Villa,A,0.536284,0.246973,0.216743,H,205,184,0.108074,2,0.437276,0.265476,0.297248,H,0,0,0,0,1
201,Newcastle,Leicester,A,0.187693,0.246965,0.565342,A,312,384,-0.207639,2,0.388217,0.270560,0.341223,H,0,1,0,0,1
202,Southampton,Tottenham,H,0.284553,0.268293,0.447154,A,259,1040,-1.390148,0,0.229065,0.245722,0.525214,A,0,0,1,0,0
203,Watford,Wolves,H,0.313804,0.276886,0.409310,A,270,315,-0.154151,0,0.396400,0.270050,0.333550,H,1,0,1,0,0
204,Man City,Everton,H,0.759124,0.145985,0.094891,H,1270,525,0.883374,0,0.561011,0.234411,0.204577,H,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Leicester,Man United,A,0.293613,0.272641,0.433746,A,384,775,-0.702220,2,0.316009,0.268300,0.415691,A,1,1,0,0,1
376,Man City,Norwich,H,0.877408,0.086146,0.036446,H,1270,110,2.446292,0,0.776981,0.136810,0.086210,H,1,1,1,0,0
377,Newcastle,Liverpool,A,0.125893,0.179847,0.694261,A,312,1090,-1.250930,2,0.245221,0.251878,0.502902,A,1,1,0,0,1
378,Southampton,Sheffield United,H,0.441345,0.271112,0.287543,H,259,110,0.856348,0,0.556736,0.235850,0.207414,H,1,1,1,0,0


In [18]:
def calc_brier_score(label_h, label_d, label_a, pred_h, pred_d, pred_a):
  return ((pred_h - label_h)**2 +(pred_d - label_d)**2 + (pred_a - label_a)**2).sum()/len(label_h)

In [19]:
brier_tm_pred = calc_brier_score(test_set['home_value'], test_set['draw_value'], test_set['away_value'], test_set['tm_pred_h'], test_set['tm_pred_d'], test_set['tm_pred_a'])
brier_tm_pred

0.5863227034381152

In [20]:
brier_b365 = calc_brier_score(test_set['home_value'], test_set['draw_value'], test_set['away_value'], test_set['b365_pr_h'], test_set['b365_pr_d'], test_set['b365_pr_a'])
brier_b365

0.5709386175611956